In [81]:
import pandas as pd

In [82]:
#import pyodbc
#print(pyodbc.drivers())


In [83]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL


connection_url = URL.create(
    "mssql+pyodbc",
    username="sa",
    password="deepthi@3",
    host="DESKTOP-7ETA4U1\SQLEXPRESS",
    database="customers",
    query={"driver": "ODBC Driver 17 for SQL Server"}
)
engine = create_engine(connection_url)

csv_files = {
    'us_customer_data.csv': 'customers',
    'order_data.csv': 'orders'
}
for csv_file, table_name in csv_files.items():
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    print(f" Uploaded {csv_file} as table `{table_name}`.")


 Uploaded us_customer_data.csv as table `customers`.
 Uploaded order_data.csv as table `orders`.


In [84]:

# Extract customers table into a DataFrame
customers_df = pd.read_sql("SELECT * FROM customers", con=engine)
print("Extracted customers table:")
customers_df.head()


Extracted customers table:


,customer_id,name,email,phone,address,registration_date,loyalty_status
0,1,Michelle Kidd,vayala@example.net,619-723-4258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold
1,2,Brad Newton,taylorcatherine@example.net,537-674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver
2,3,Larry Torres,dsanchez@example.net,810-256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze
3,4,Kimberly Price,jessicaknight@example.com,423-222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold
4,5,Matthew Phillips,qwilliams@example.com,220-763-3522,"2274 Williams Heights Suite 895, Andersonhaven...",03-02-2024,Gold


In [85]:
orders_df=pd.read_sql("select * from orders",con=engine)
print("extracted orders table:")
orders_df.head()

extracted orders table:


,order_id,customer_id,order_date,order_amount,order_status,product_category
0,892a07a4-d252-4775-85e0-73077143e1c6,966,2024-11-27,317.64,Cancelled,Home & Garden
1,ae160758-e187-47b2-9350-032f88f55491,345,2023-03-27,645.87,Completed,Home & Garden
2,7c50456e-6123-45cc-aa19-128bef3754d6,503,2024-03-31,880.86,Pending,Clothing
3,c788b56b-3716-4cd9-a827-d4dc401ba00c,385,2023-08-09,876.83,Cancelled,Home & Garden
4,925ab5b1-adb1-4302-a70b-1c2db724e02b,817,2023-04-19,264.53,Pending,Home & Garden


In [86]:
df = pd.merge(customers_df, orders_df, on='customer_id', how='inner')

print("Unified Customer View created. Preview:")
df.head()


Unified Customer View created. Preview:


,customer_id,name,email,phone,address,registration_date,loyalty_status,order_id,order_date,order_amount,order_status,product_category
0,2,Brad Newton,taylorcatherine@example.net,537-674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,209c024e-6c32-49fa-acd1-2501bd19c1cd,2023-10-30,486.53,Pending,Toys
1,2,Brad Newton,taylorcatherine@example.net,537-674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,3f263091-da18-48a6-8b00-c179e1b1a81a,2024-08-29,781.22,Pending,Books
2,3,Larry Torres,dsanchez@example.net,810-256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,63d7870e-617e-4f49-ae9b-b893e635e43f,2023-04-05,454.07,Pending,Toys
3,4,Kimberly Price,jessicaknight@example.com,423-222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,cbe7b5ef-013f-4b4f-9ac4-3b510075080a,2023-08-12,259.07,Cancelled,Electronics
4,4,Kimberly Price,jessicaknight@example.com,423-222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,21209e05-078a-402d-a450-dd310595e5f0,2024-08-27,117.03,Completed,Electronics


In [87]:
df.isnull().sum()  

customer_id            0
name                   0
email                106
phone                 35
address                0
registration_date      0
loyalty_status         0
order_id               0
order_date             0
order_amount           0
order_status           0
product_category       0
dtype: int64

In [88]:
df.nunique()  # Check for unique values in the merged DataFrame

customer_id           868
name                  859
email                 763
phone                 852
address               868
registration_date     513
loyalty_status          3
order_id             2000
order_date            688
order_amount         1974
order_status            3
product_category        5
dtype: int64

In [89]:
prefixes = ['Mr.', 'Mr', 'Miss.', 'Miss', 'Mrs.', 'Mrs', 'Dr.', 'Dr']
 
def clean_name(name):
    parts = name.split()
    if len(parts) == 2:
        return ' '.join(parts)
    elif len(parts) == 4:
        return ' '.join(parts[1:-1])
    elif len(parts) == 3:
        if parts[0] in prefixes:
            return ' '.join(parts[1:])
        else:
            return ' '.join(parts[:-1])
    else:
        return name  
 
df['name'] = df['name'].apply(clean_name)

In [90]:
print(df)

      customer_id            name                        email         phone  \
0               2     Brad Newton  taylorcatherine@example.net  537-674-1158   
1               2     Brad Newton  taylorcatherine@example.net  537-674-1158   
2               3    Larry Torres         dsanchez@example.net  810-256-4505   
3               4  Kimberly Price    jessicaknight@example.com  423-222-9779   
4               4  Kimberly Price    jessicaknight@example.com  423-222-9779   
...           ...             ...                          ...           ...   
1995          997    Jodi Simpson           eric24@example.org  483-625-2940   
1996          998   Crystal Brown         pshaffer@example.net  390-747-3088   
1997          998   Crystal Brown         pshaffer@example.net  390-747-3088   
1998          999  Gregory Duarte    caitlindunlap@example.org  257-409-8196   
1999         1000     Jason Walls        michael42@example.com  692-333-6880   

                                       

In [91]:
df.sample(n=10)

,customer_id,name,email,phone,address,registration_date,loyalty_status,order_id,order_date,order_amount,order_status,product_category
83,53,Phillip Brown,yjackson@example.org,933-317-4041,"043 Bryant Avenue, Williamsview, WA 36043",30-05-2024,Bronze,2024dd11-1154-4270-a7a7-c74ccf76100e,2024-03-12,882.16,Pending,Clothing
1748,879,Mitchell Hodges,chelsey31@example.com,898-582-7024,"135 Destiny Fords Suite 513, Mcknightmouth, MT...",26-03-2025,Gold,2d4c0a46-620a-4b2e-8a09-af0dd35086dd,2024-10-19,458.54,Pending,Home & Garden
1639,826,Benjamin Hawkins,changjulie@example.org,307-850-6694,"0488 Moore Ridges Suite 286, Lake Rachelhaven,...",11-07-2023,Silver,8c753178-13d5-4830-8c0a-f037ad46fc83,2024-06-12,980.22,Pending,Home & Garden
908,461,Dennis Wyatt,timothy80@example.org,759-165-8568,"08014 Patricia Hollow, Lake Martinshire, AR 03056",12-12-2023,Bronze,fa2903b4-c2b5-4d42-acde-1f8657caae47,2024-09-13,274.96,Completed,Home & Garden
1432,729,Jessica Wiley,sharon92@example.org,702-224-5359,"77702 Allen Motorway Suite 904, Alexanderside,...",21-06-2023,Silver,14d75c17-f7bb-467a-a8f7-60bd12afd626,2023-08-19,630.91,Completed,Electronics
1893,945,Cynthia Parks,simslarry@example.com,129-878-7837,"7949 Thomas Harbor Suite 743, Nancybury, WI 30420",05-12-2024,Gold,b6170700-f407-4bc0-8288-7c88653a489d,2023-10-20,416.10,Pending,Clothing
1211,612,Ricky Levine,egomez@example.org,624-122-3395,"79039 Daniel Drive, New Stacey, TX 69894",17-10-2023,Silver,01ac7dff-ac61-482c-8e4c-218142c39a74,2024-08-30,287.05,Pending,Electronics
80,51,David Garza,None,352-638-6388,"USS Williams, FPO AA 77634",24-10-2023,Silver,56fdfc34-5f87-4419-afb3-c66702c8510b,2023-09-10,919.01,Cancelled,Electronics
1724,870,Ashley Romero,bakerbrittney@example.com,459-161-6754,"PSC 6583, Box 4321, APO AP 18388",09-05-2024,Bronze,4644d86a-1f12-4f7d-b89e-0108256b18a0,2023-09-26,365.22,Pending,Clothing
849,436,Daniel Nguyen,jenniferowens@example.org,131-376-8888,"5014 Lori Hollow Suite 764, North Patriciaview...",13-01-2024,Gold,b4f7e383-6a11-464d-9f4b-c1cbb06f3a3a,2024-10-30,316.35,Completed,Books


In [92]:
#df.to_csv('hasini_data.csv')

In [93]:
name_idx = df.columns.get_loc('name')

new_cols = df['name'].str.strip().str.split(' ', n=1, expand=True)
new_cols.columns = ['First_Name', 'Last_Name']


for i, col_name in enumerate(['First_Name', 'Last_Name']):
    df.insert(name_idx + 1 + i, col_name, new_cols[col_name])

In [94]:
print(df)

      customer_id            name First_Name Last_Name  \
0               2     Brad Newton       Brad    Newton   
1               2     Brad Newton       Brad    Newton   
2               3    Larry Torres      Larry    Torres   
3               4  Kimberly Price   Kimberly     Price   
4               4  Kimberly Price   Kimberly     Price   
...           ...             ...        ...       ...   
1995          997    Jodi Simpson       Jodi   Simpson   
1996          998   Crystal Brown    Crystal     Brown   
1997          998   Crystal Brown    Crystal     Brown   
1998          999  Gregory Duarte    Gregory    Duarte   
1999         1000     Jason Walls      Jason     Walls   

                            email         phone  \
0     taylorcatherine@example.net  537-674-1158   
1     taylorcatherine@example.net  537-674-1158   
2            dsanchez@example.net  810-256-4505   
3       jessicaknight@example.com  423-222-9779   
4       jessicaknight@example.com  423-222-9779 

In [95]:
df.isnull().sum()

customer_id            0
name                   0
First_Name             0
Last_Name              0
email                106
phone                 35
address                0
registration_date      0
loyalty_status         0
order_id               0
order_date             0
order_amount           0
order_status           0
product_category       0
dtype: int64

In [96]:
df['email']=df['email'].fillna( df['First_Name'].str.lower().str.replace(r'\s+', '', regex=True)
+ df['Last_Name'].str.lower().str.replace(r'\s+', '', regex=True) + '@example.com')

In [97]:
df[df['customer_id']==499]

,customer_id,name,First_Name,Last_Name,email,phone,address,registration_date,loyalty_status,order_id,order_date,order_amount,order_status,product_category
976,499,Shane Warner,Shane,Warner,shanewarner@example.com,335-327-9805,"8106 Cory Green Apt. 982, South Katherinefort,...",28-06-2024,Bronze,1356b72c-7d27-4776-8ebc-6f522ada3dce,2023-11-10,512.74,Completed,Electronics
977,499,Shane Warner,Shane,Warner,shanewarner@example.com,335-327-9805,"8106 Cory Green Apt. 982, South Katherinefort,...",28-06-2024,Bronze,7ae210fa-af41-46b4-b86f-febb30601d84,2023-05-24,296.13,Completed,Clothing
978,499,Shane Warner,Shane,Warner,shanewarner@example.com,335-327-9805,"8106 Cory Green Apt. 982, South Katherinefort,...",28-06-2024,Bronze,44205198-92c0-4a49-a819-dd66e25507fb,2024-10-31,834.76,Cancelled,Home & Garden
979,499,Shane Warner,Shane,Warner,shanewarner@example.com,335-327-9805,"8106 Cory Green Apt. 982, South Katherinefort,...",28-06-2024,Bronze,83fdd478-5985-4c9e-8788-bcf370173e88,2023-12-07,810.97,Cancelled,Clothing
980,499,Shane Warner,Shane,Warner,shanewarner@example.com,335-327-9805,"8106 Cory Green Apt. 982, South Katherinefort,...",28-06-2024,Bronze,d20cb41e-86ed-49b4-8541-4dc0610b4561,2024-03-12,749.44,Completed,Home & Garden
981,499,Shane Warner,Shane,Warner,shanewarner@example.com,335-327-9805,"8106 Cory Green Apt. 982, South Katherinefort,...",28-06-2024,Bronze,12413a23-9078-456e-b5d4-424591284025,2023-12-27,554.20,Cancelled,Toys
982,499,Shane Warner,Shane,Warner,shanewarner@example.com,335-327-9805,"8106 Cory Green Apt. 982, South Katherinefort,...",28-06-2024,Bronze,b89d15be-3b7a-43b4-8b9a-4da5cf982a2c,2023-10-03,913.90,Cancelled,Books
983,499,Shane Warner,Shane,Warner,shanewarner@example.com,335-327-9805,"8106 Cory Green Apt. 982, South Katherinefort,...",28-06-2024,Bronze,56cdd2ab-d97a-42b7-9f68-65ae1cc599dd,2024-01-03,769.78,Pending,Clothing


In [98]:
mapping_dict={'Gold':0,'Silver':1,'Bronze':2}
df['loyalty_range']=df['loyalty_status'].map(mapping_dict)

In [99]:
print(df)

      customer_id            name First_Name Last_Name  \
0               2     Brad Newton       Brad    Newton   
1               2     Brad Newton       Brad    Newton   
2               3    Larry Torres      Larry    Torres   
3               4  Kimberly Price   Kimberly     Price   
4               4  Kimberly Price   Kimberly     Price   
...           ...             ...        ...       ...   
1995          997    Jodi Simpson       Jodi   Simpson   
1996          998   Crystal Brown    Crystal     Brown   
1997          998   Crystal Brown    Crystal     Brown   
1998          999  Gregory Duarte    Gregory    Duarte   
1999         1000     Jason Walls      Jason     Walls   

                            email         phone  \
0     taylorcatherine@example.net  537-674-1158   
1     taylorcatherine@example.net  537-674-1158   
2            dsanchez@example.net  810-256-4505   
3       jessicaknight@example.com  423-222-9779   
4       jessicaknight@example.com  423-222-9779 

In [100]:
name_idx = df.columns.get_loc('loyalty_status')#cols.index('loyalty_status')
col = df.pop('loyalty_range')
df.insert(name_idx+1, 'loyalty_range', col)

print(df)

      customer_id            name First_Name Last_Name  \
0               2     Brad Newton       Brad    Newton   
1               2     Brad Newton       Brad    Newton   
2               3    Larry Torres      Larry    Torres   
3               4  Kimberly Price   Kimberly     Price   
4               4  Kimberly Price   Kimberly     Price   
...           ...             ...        ...       ...   
1995          997    Jodi Simpson       Jodi   Simpson   
1996          998   Crystal Brown    Crystal     Brown   
1997          998   Crystal Brown    Crystal     Brown   
1998          999  Gregory Duarte    Gregory    Duarte   
1999         1000     Jason Walls      Jason     Walls   

                            email         phone  \
0     taylorcatherine@example.net  537-674-1158   
1     taylorcatherine@example.net  537-674-1158   
2            dsanchez@example.net  810-256-4505   
3       jessicaknight@example.com  423-222-9779   
4       jessicaknight@example.com  423-222-9779 

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        2000 non-null   int64  
 1   name               2000 non-null   object 
 2   First_Name         2000 non-null   object 
 3   Last_Name          2000 non-null   object 
 4   email              2000 non-null   object 
 5   phone              1965 non-null   object 
 6   address            2000 non-null   object 
 7   registration_date  2000 non-null   object 
 8   loyalty_status     2000 non-null   object 
 9   loyalty_range      2000 non-null   int64  
 10  order_id           2000 non-null   object 
 11  order_date         2000 non-null   object 
 12  order_amount       2000 non-null   float64
 13  order_status       2000 non-null   object 
 14  product_category   2000 non-null   object 
dtypes: float64(1), int64(2), object(12)
memory usage: 234.5+ KB


In [103]:
import re
import numpy as np

def clean_ph(ph):
    if pd.isna(ph):
        return np.nan
    new_phn=re.sub(r'\D','',str(ph))
    return new_phn if new_phn else np.nan
df['phone']=df['phone'].apply(clean_ph)



In [ ]:
df['phone'] = df['phone'].astype(str)
#df['phone']=df['phone'].fillna('0000000000')

In [119]:
df['phone'].str.len().value_counts()

phone
10    2000
Name: count, dtype: int64

In [117]:
df.loc[df['phone'].str.len() != 10, 'phone'] = '0000000000'

In [121]:
def format_us_phone(ph):
    if len(ph) == 10 and ph.isdigit():
        return f'+1 {ph[:3]}-{ph[3:6]}-{ph[6:]}'
    return ph 

# Apply formatting
df['phone'] = df['phone'].apply(format_us_phone)


In [124]:
df.to_csv('temp.csv')

In [125]:
df.to_sql('unified_customer_view',con=engine, if_exists='replace', index=False)
print(" Uploaded temp.csv as table unified_customer_view")

 Uploaded temp.csv as table unified_customer_view
